In [1]:
# Gemini limits
# Gemini 2.0 Flash - 15 per minute, 1,500 per day, 1,000,000 tokens per minute
# Gemini 1.5 Flash - 15 per minute, 1,500 per day, 1,000,000 tokens per minute

In [2]:
%%capture
%pip install -U google-generativeai
%pip install -U google-genai
%pip install -U openai

In [3]:
import os
from os import listdir
from os.path import isfile, join
import json

In [6]:
from google import genai
import json

with open('keys.json', encoding='utf-8') as f:
    api_keys = json.load(f)

google_api_key = api_keys['Gemini']
# openrouter_api_key = api_keys['OpenRouter']

In [7]:
google_client = genai.Client(api_key=google_api_key)

In [8]:
# from openai import OpenAI

# client = OpenAI(
#   base_url="https://openrouter.ai/api/v1",
#   api_key=openrouter_api_key,
# )

In [9]:
import time
from google.genai.types import Tool, ToolCodeExecution, GenerateContentConfig

code_execution_tool = Tool(
    code_execution=ToolCodeExecution()
)

def GetGeminiResponses(instructions, iterations=1, model="gemini-2.0-flash", history = {}):
    results = {}
    try:
        for id, instruction in instructions.items():
            print(id)
            results[id] = []
            if id in history.keys(): 
                print('Exists:',len(history[id]))
                count = len(history[id])
            else:
                count = 0
            for _ in range(count,iterations):
                response = google_client.models.generate_content(
                    model=model,
                    contents=instruction,
                    config=GenerateContentConfig(
                        tools=[code_execution_tool],
                        temperature=0, # determmines randomness. The lower the value, the less random is the text.
                    )
                )
                results[id].append(response.text)
                time.sleep(6)
        return results, None
    except Exception as error:
        print('Error:',error)
        return results, error

In [10]:
def GetScores(file, model, instructions):
    iterations = 1

    print(file)

    while True:
        try:
            with open(f'Scores/LlmEvaluatorAnswers/{model}_{file}', encoding='utf-8') as f:
                scores = json.load(f)
        except:
            scores = {}

        if len(scores) == len(instructions) and all([len(score) >= iterations for _, score in scores.items() ]): break

        instructions_check = { inst:instructions[inst] for inst in instructions.keys() if inst not in scores.keys() or len(scores[inst]) < iterations}
        history = { inst:scores[inst] for inst in scores.keys() if len(scores[inst]) < iterations}

        print(instructions_check.keys())

        result, error = GetGeminiResponses(instructions_check, iterations=iterations, model=model, history=history)

        final = {k: scores.get(k, []) + result.get(k, []) for k in set(scores) | set(result)}

        with open(f'Scores/LlmEvaluatorAnswers/{model}_{file}', 'wt', encoding='utf-8') as f:
            json.dump(final, f, ensure_ascii=False, indent=4)

        if error != None:
            if error.status == 'RESOURCE_EXHAUSTED': return error.status

    return None


In [14]:
path = 'instructions/SupervisedLlmEvaluation/'
model = 'gemini-2.0-flash'

onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]

for file in onlyfiles:
    with open(path+file, encoding='utf-8') as f:
        instructions = json.load(f)
    status = GetScores(file, model, instructions)
    if status != None: break

In [15]:
# def GetResponses(instructions, iterations=1, model="deepseek/deepseek-r1:free", history = {}):
#     results = {}
#     try:
#         for id, instruction in instructions.items():
#             print(id)
#             results[id] = []
#             if id in history.keys(): 
#                 print('Exists:',len(history[id]))
#                 count = len(history[id])
#             else:
#                 count = 0
#             for _ in range(count,iterations):
#                 response = client.chat.completions.create(
#                     extra_body={},
#                     model=model,
#                     messages=[
#                         {
#                             "role": "user",
#                             "content": instruction
#                         }
#                     ]
#                 )
#                 results[id].append(response.choices[0].message.content)
#                 time.sleep(3)
#         return results
#     except Exception as error:
#         print('Error:', response.error['message'])
#         return results
